In [166]:
import json
import pandas as pd
from shapely.ops import cascaded_union
from pandas.io.json import json_normalize
from shapely.geometry import Point, Polygon
import geopandas as gpd
from folium import Map, Marker, GeoJson, features
from h3 import h3
import requests
from zipfile import ZipFile
import xmltodict, io

In [195]:
import time
ts = round(time.time())
print(ts)

1587334853


In [199]:
resp = requests.get("http://imrad.sire.gov.co:8080/radar/reflectividad.kmz?rand=%s" % ts)
k = zipfile.ZipFile(io.BytesIO(resp.content))
k.extractall('reflec')
km = k.open('doc.kml', 'r').read()
o = xmltodict.parse(km)
img_radar = o["kml"]["Folder"]["GroundOverlay"]["Icon"]["href"]
o

OrderedDict([('kml',
              OrderedDict([('@xmlns', 'http://earth.google.com/kml/2.2'),
                           ('Folder',
                            OrderedDict([('name',
                                          '1399BOG-Intensity (Horizontal)-Sun Apr 19 22:44:11 2020 GMT'),
                                         ('ExtendedData',
                                          OrderedDict([('@xmlns:edge',
                                                        'http://eecradar.com'),
                                                       ('edge:Moment',
                                                        'Intensity (Horizontal)'),
                                                       ('edge:MomentAbbreviation',
                                                        'ZH')])),
                                         ('lookAt',
                                          OrderedDict([('longitude',
                                                        '-74.11397552490234'),

In [197]:
from PIL import Image 
Image.open('reflec/%s' % img_radar).show()
iconmm = features.CustomIcon('reflec/%s' % img_radar, icon_size=(500,500))

In [198]:
## Just use hexagon resolution 7 to be able to get weather data 
with open('source/map_bogota_hex7.json') as f:
    bogota_hex7 = json.load(f)
    
mapa_bogota = Map(location= [4.626853707360377,-74.0682993486343], zoom_start=11, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )
Marker([4.626853707360377,-74.0682993486343], icon=iconmm).add_to(mapa_bogota)

GeoJson(
        bogota_hex7,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'red',
            'weight': 2,
            'fillOpacity': 0
        }, 
        name = "Hexagons_b" 
    ).add_to(mapa_bogota)

mapa_bogota